In [1]:
import sys
sys.path.append("/home/chaoqiy2/github/PyHealth-OMOP")

# 1. load dataset

In [2]:
from pyhealth.datasets import MIMIC3BaseDataset
base_ds = MIMIC3BaseDataset(root="/srv/local/data/physionet.org/files/mimiciii/1.4", flag="dev")

1. loaded the based dataset from cache


# 2. drug recommendation task

In [3]:
from pyhealth.tasks import DrugRecDataset
drugrec_ds = DrugRecDataset(base_ds)

----- preparing code mappings -----
source loaded from ./data cache
source loaded from ./data cache
source loaded from ./data cache
source loaded from ./data cache
mapping finished: RxNorm -> ATC4
mapping finished: ATC4 -> RxNorm
load time: 2.8688981533050537s
-----------------------------------------
1. finish cleaning the dataset for drug recommendation task
2. tokenized the medical codes


In [9]:
from pyhealth.data.split import split_by_pat
train_loader, val_loader, test_loader = split_by_pat(drugrec_ds, [0.8, 0.1, 0.1], 64)

1. finish data splitting
2. generate train / val / test data loaders


# 3. initialize ML models

In [5]:
from pyhealth.models import RetainDrugRec
Retain = RetainDrugRec(
    voc_size = drugrec_ds.voc_size,
    tokenizers = drugrec_ds.tokenizers,
    emb_dim = 64,
)